<a href="https://colab.research.google.com/github/dev2081/Dev-Singh-Khadayat/blob/main/scrapping_on_worldometer_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests pandas lxml

from bs4 import BeautifulSoup
import requests
import pandas as pd


url = 'https://www.worldometers.info/coronavirus/'


response = requests.get(url)

# Create the BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')


covid_dict = {}
for div in soup.find_all("div", id="maincounter-wrap"):
    content_div = div.find("div", {"class": "maincounter-number"})
    if content_div:
        covid_dict[div.find("h1").text.replace(":", "").strip()] = content_div.find("span").text.strip()

                                    # 2. Extract table data
table = soup.find('table', id="main_table_countries_today")
table_head = [th.text.replace("\n", "").replace("\xa0", "") for th in table.find('thead').find_all('th')]
table_rows = []

for row in table.find('tbody').find_all('tr'):
    tds = [td.text.replace("\n", "").replace("\xa0", "").strip() for td in row.find_all('td')]
    if tds:
        table_rows.append(tds)

                                    # 3. Create Pandas DataFrame
df = pd.DataFrame(table_rows, columns=table_head)
total_cases = covid_dict['Coronavirus Cases']
total_deaths = covid_dict['Deaths']
total_recovered = covid_dict['Recovered']

print("Total Cases:", total_cases)
print("Total Deaths:", total_deaths)
print("Total Recovered:", total_recovered)
print("Table Data:\n", df)
usa_data = df[df['Country,Other'] == 'USA']

print("USA Data:\n", usa_data)

Total Cases: 704,753,890
Total Deaths: 7,010,681
Total Recovered: 675,619,811
Table Data:
        #   Country,Other   TotalCases NewCases TotalDeaths NewDeaths  \
0          North America  131,889,132            1,695,941             
1                   Asia  221,500,265            1,553,662             
2                 Europe  253,406,198            2,101,824             
3          South America   70,200,879            1,367,332             
4                Oceania   14,895,771               33,015             
..   ...             ...          ...      ...         ...       ...   
234  227         Tokelau           80                                  
235  228    Vatican City           29                                  
236  229  Western Sahara           10                    1             
237  230      MS Zaandam            9                    2             
238  231           China      503,302                5,272             

    TotalRecovered NewRecovered ActiveCases 

In [ ]:
df

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,North America,"131,889,132",,"1,695,941",,"127,665,129",+350,"2,528,062","6,095",...,,,,North America,,,,,,
1,,Asia,"221,500,265",,"1,553,662",,"205,673,091",,"14,273,512","14,733",...,,,,Asia,,,,,,
2,,Europe,"253,406,198",,"2,101,824",,"248,754,104",+474,"2,550,270","4,453",...,,,,Europe,,,,,,
3,,South America,"70,200,879",,"1,367,332",,"66,683,585",,"2,149,962","8,953",...,,,,South America,,,,,,
4,,Oceania,"14,895,771",,"33,015",,"14,752,388",,"110,368",31,...,,,,Australia/Oceania,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,227,Tokelau,80,,,,,,80,,...,,,"1,378",Australia/Oceania,17,,,,,"58,055"
235,228,Vatican City,29,,,,29,,0,,...,,,799,Europe,28,,,,,
236,229,Western Sahara,10,,1,,9,,0,,...,,,"626,161",Africa,"62,616","626,161",,,,
237,230,MS Zaandam,9,,2,,7,,0,,...,,,,,,,,,,


In [ ]:
df = df.drop('NewCases', axis=1)

In [ ]:
df

,#,"Country,Other",TotalCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,North America,"131,889,132","1,695,941",,"127,665,129",+350,"2,528,062","6,095",,...,,,,North America,,,,,,
1,,Asia,"221,500,265","1,553,662",,"205,673,091",,"14,273,512","14,733",,...,,,,Asia,,,,,,
2,,Europe,"253,406,198","2,101,824",,"248,754,104",+474,"2,550,270","4,453",,...,,,,Europe,,,,,,
3,,South America,"70,200,879","1,367,332",,"66,683,585",,"2,149,962","8,953",,...,,,,South America,,,,,,
4,,Oceania,"14,895,771","33,015",,"14,752,388",,"110,368",31,,...,,,,Australia/Oceania,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,227,Tokelau,80,,,,,80,,"58,055",...,,,"1,378",Australia/Oceania,17,,,,,"58,055"
235,228,Vatican City,29,,,29,,0,,"36,295",...,,,799,Europe,28,,,,,
236,229,Western Sahara,10,1,,9,,0,,16,...,,,"626,161",Africa,"62,616","626,161",,,,
237,230,MS Zaandam,9,2,,7,,0,,,...,,,,,,,,,,


In [ ]:
df = df.drop('NewDeaths', axis=1)

In [ ]:
df

,#,"Country,Other",TotalCases,TotalDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,North America,"131,889,132","1,695,941","127,665,129",+350,"2,528,062","6,095",,,,,,North America,,,,,,
1,,Asia,"221,500,265","1,553,662","205,673,091",,"14,273,512","14,733",,,,,,Asia,,,,,,
2,,Europe,"253,406,198","2,101,824","248,754,104",+474,"2,550,270","4,453",,,,,,Europe,,,,,,
3,,South America,"70,200,879","1,367,332","66,683,585",,"2,149,962","8,953",,,,,,South America,,,,,,
4,,Oceania,"14,895,771","33,015","14,752,388",,"110,368",31,,,,,,Australia/Oceania,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,227,Tokelau,80,,,,80,,"58,055",,,,"1,378",Australia/Oceania,17,,,,,"58,055"
235,228,Vatican City,29,,29,,0,,"36,295",,,,799,Europe,28,,,,,
236,229,Western Sahara,10,1,9,,0,,16,2,,,"626,161",Africa,"62,616","626,161",,,,
237,230,MS Zaandam,9,2,7,,0,,,,,,,,,,,,,


In [ ]:
df = df.drop('New Cases/1M pop', axis=1)

In [ ]:
df

,#,"Country,Other",TotalCases,TotalDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Deaths/1M pop,Active Cases/1M pop
0,,North America,"131,889,132","1,695,941","127,665,129",+350,"2,528,062","6,095",,,,,,North America,,,,,
1,,Asia,"221,500,265","1,553,662","205,673,091",,"14,273,512","14,733",,,,,,Asia,,,,,
2,,Europe,"253,406,198","2,101,824","248,754,104",+474,"2,550,270","4,453",,,,,,Europe,,,,,
3,,South America,"70,200,879","1,367,332","66,683,585",,"2,149,962","8,953",,,,,,South America,,,,,
4,,Oceania,"14,895,771","33,015","14,752,388",,"110,368",31,,,,,,Australia/Oceania,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,227,Tokelau,80,,,,80,,"58,055",,,,"1,378",Australia/Oceania,17,,,,"58,055"
235,228,Vatican City,29,,29,,0,,"36,295",,,,799,Europe,28,,,,
236,229,Western Sahara,10,1,9,,0,,16,2,,,"626,161",Africa,"62,616","626,161",,,
237,230,MS Zaandam,9,2,7,,0,,,,,,,,,,,,


In [ ]:
df = df.drop('New Deaths/1M pop', axis=1)

In [ ]:
df

,#,"Country,Other",TotalCases,TotalDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,Active Cases/1M pop
0,,North America,"131,889,132","1,695,941","127,665,129",+350,"2,528,062","6,095",,,,,,North America,,,,
1,,Asia,"221,500,265","1,553,662","205,673,091",,"14,273,512","14,733",,,,,,Asia,,,,
2,,Europe,"253,406,198","2,101,824","248,754,104",+474,"2,550,270","4,453",,,,,,Europe,,,,
3,,South America,"70,200,879","1,367,332","66,683,585",,"2,149,962","8,953",,,,,,South America,,,,
4,,Oceania,"14,895,771","33,015","14,752,388",,"110,368",31,,,,,,Australia/Oceania,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,227,Tokelau,80,,,,80,,"58,055",,,,"1,378",Australia/Oceania,17,,,"58,055"
235,228,Vatican City,29,,29,,0,,"36,295",,,,799,Europe,28,,,
236,229,Western Sahara,10,1,9,,0,,16,2,,,"626,161",Africa,"62,616","626,161",,
237,230,MS Zaandam,9,2,7,,0,,,,,,,,,,,


In [ ]:
for column in ['TotalCases', 'TotalDeaths', 'TotalRecovered']:
    df[column] = df[column].astype(str).str.replace(',', '')
    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(int)

In [ ]:
df

,#,"Country,Other",TotalCases,TotalDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,Active Cases/1M pop
0,,North America,131889132,1695941,127665129,+350,"2,528,062","6,095",,,,,,North America,,,,
1,,Asia,221500265,1553662,205673091,,"14,273,512","14,733",,,,,,Asia,,,,
2,,Europe,253406198,2101824,248754104,+474,"2,550,270","4,453",,,,,,Europe,,,,
3,,South America,70200879,1367332,66683585,,"2,149,962","8,953",,,,,,South America,,,,
4,,Oceania,14895771,33015,14752388,,"110,368",31,,,,,,Australia/Oceania,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,227,Tokelau,80,0,0,,80,,"58,055",,,,"1,378",Australia/Oceania,17,,,"58,055"
235,228,Vatican City,29,0,29,,0,,"36,295",,,,799,Europe,28,,,
236,229,Western Sahara,10,1,9,,0,,16,2,,,"626,161",Africa,"62,616","626,161",,
237,230,MS Zaandam,9,2,7,,0,,,,,,,,,,,


In [ ]:

df_sorted = df.sort_values(by=['TotalCases'], ascending=False)
print(df_sorted[['Country,Other', 'TotalCases']].head(10))

    Country,Other  TotalCases
7           World   704753890
2          Europe   253406198
1            Asia   221500265
0   North America   131889132
8             USA   111820082
3   South America    70200879
9           India    45035393
10         France    40138560
11        Germany    38828995
12         Brazil    38743918


In [ ]:
df1 = df.sort_values(by=['TotalCases'], ascending=False).iloc[4:18]
df1


,#,"Country,Other",TotalCases,TotalDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,Active Cases/1M pop
8,1,USA,111820082,1219487,109814428,,"786,167",940,"333,985","3,642","1,186,851,502","3,544,901","334,805,269",North America,3,275,0,"2,348"
3,,South America,70200879,1367332,66683585,,"2,149,962","8,953",,,,,,South America,,,,
9,2,India,45035393,533570,0,N/A,N/A,N/A,"32,016",379,"935,879,495","665,334","1,406,631,776",Asia,31,"2,636",2,0.4
10,3,France,40138560,167642,39970918,,0,,"612,013","2,556","271,490,188","4,139,547","65,584,518",Europe,2,391,0,
11,4,Germany,38828995,183027,38240600,,"405,368",N/A,"462,891","2,182","122,332,384","1,458,359","83,883,596",Europe,2,458,1,"4,833"
12,5,Brazil,38743918,711380,36249161,,"1,783,377",N/A,"179,908","3,303","63,776,166","296,146","215,353,593",South America,6,303,3,"8,281"
13,6,S. Korea,34571873,35934,34535939,,0,,"673,523",700,"15,804,065","307,892","51,329,899",Asia,1,"1,428",3,
14,7,Japan,33803572,74694,0,N/A,N/A,N/A,"269,169",595,"100,414,883","799,578","125,584,838",Asia,4,"1,681",1,"95,582"
15,8,Italy,26723249,196487,26361218,,"165,544",22,"443,445","3,261","281,126,449","4,665,010","60,262,770",Europe,2,307,0,"2,747"
16,9,UK,24910387,232112,24678275,,0,N/A,"363,666","3,389","522,526,476","7,628,357","68,497,907",Europe,3,295,0,


In [ ]:
import plotly.express as px
fig = px.bar(df1, x='Country,Other', y='TotalCases')
fig.show()